In [ ]:
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path
import os

# load environment variables from .env file for project
dotenv_path = Path('../.env')
load_dotenv(dotenv_path=dotenv_path)

data_directory = os.getenv("OUTPUT_DIRECTORY")
feature_directory = os.path.join(data_directory, "features")

In [ ]:
lidl_df = pd.read_parquet(os.path.join(data_directory, "ssi_lidl_revenue.parquet"), engine="pyarrow")
lidl_df.rename(columns={"bg_number": "supermarket_id", "ean_name": "receipt_text"}, inplace=True)
lidl_df.receipt_text = lidl_df.receipt_text.str.replace('[^0-9a-zA-Z.,-/ ]', '', regex=True).str.lstrip().str.rstrip().str.lower()
lidl_df.head()

## Unique products 

In [ ]:
unique_products = lidl_df[["receipt_text", "ean_number"]].nunique()
unique_products.to_latex("../Notebooks/tables/lidl_unique_products_total.tex")
unique_products

In [ ]:
unique_products_per_month = lidl_df.groupby(by=["year_month"])[["receipt_text", "ean_number"]].nunique()
unique_products_per_month.to_latex("../Notebooks/tables/lidl_unique_product_per_month.tex")
unique_products_per_month

In [ ]:
unique_products_per_month.plot.bar(figsize=(15,10)).figure.savefig("../Notebooks/plots/lidl_unique_products_per_month.png")

In [ ]:
unique_products_per_coicop = lidl_df.groupby(by=["coicop_level_1"])[["receipt_text", "ean_number"]].nunique()
unique_products_per_coicop.to_latex("../Notebooks/tables/lidl_unique_products_coicop_level_1.tex")
unique_products_per_coicop

In [ ]:
unique_products_per_coicop.plot.bar().figure.savefig("../Notebooks/plots/lidl_unique_products_per_coicop.png")

In [ ]:
texts_per_ean.receipt_text.value_counts().sort_index()

In [ ]:
texts_per_ean = lidl_df.groupby(by="ean_number")["receipt_text"].nunique()
texts_per_ean = texts_per_ean.reset_index()
receipt_text_counts = texts_per_ean.receipt_text.value_counts()
receipt_text_counts = receipt_text_counts.sort_index()
receipt_text_counts = np.log(receipt_text_counts)
receipt_text_counts

In [ ]:
import numpy as np

texts_per_ean = lidl_df.groupby(by="ean_number")["receipt_text"].nunique()
texts_per_ean = texts_per_ean.reset_index()
receipt_text_counts = texts_per_ean.receipt_text.value_counts()
receipt_text_counts = receipt_text_counts.sort_index()
receipt_text_counts = np.log(receipt_text_counts)

receipt_text_counts.plot.bar(y="receipt_text", rot=90, figsize=(10,10), xlabel="Number of receipt texts per EAN", ylabel="Log(count)").figure.savefig("../Notebooks/plots/lidl_hist_receipt_texts_per_ean.png")


## New/removed Products per Month

In [ ]:
from text_analysis import compare_receipt_texts_per_period

lidl_new_products_df = compare_receipt_texts_per_period(lidl_df, "year_month", "receipt_text")
lidl_new_products_df.head()

In [ ]:
lidl_new_products_df.columns

In [ ]:
from text_analysis import series_to_set

grouped_texts_per_month = lidl_new_products_df.groupby(by="year_month")["receipt_text"].apply(series_to_set)
grouped_texts_per_month = grouped_texts_per_month.reset_index()
grouped_texts_per_month 

In [ ]:
grouped_eans_per_month = lidl_new_products_df.groupby(by="year_month")["ean_number"].apply(series_to_set)
grouped_eans_per_month = grouped_eans_per_month.reset_index()
grouped_eans_per_month

In [ ]:
grouped_texts_eans_per_month = grouped_texts_per_month.merge(grouped_eans_per_month, on="year_month")
grouped_texts_eans_per_month

In [ ]:
grouped_texts_eans_per_month = grouped_texts_eans_per_month.set_index("year_month")
grouped_texts_eans_per_month

In [ ]:
grouped_texts_eans_per_month["receipt_text_lagged"] = grouped_texts_eans_per_month["receipt_text"].shift(1)
grouped_texts_eans_per_month["ean_number_lagged"] = grouped_texts_eans_per_month["ean_number"].shift(1)
grouped_texts_eans_per_month

In [ ]:
from typing import Optional

def intersection(left_column: Optional[set], right_column: Optional[set]) -> Optional[set]:
    if not left_column or not right_column:
        return None    
    return left_column.intersection(right_column)

def introduced_products(left_column: Optional[set], right_column: Optional[set]) -> Optional[set]:
    if not left_column or not right_column:
        return None    
    return left_column.difference(right_column)

def removed_products(left_column: Optional[set], right_column: Optional[set]) -> Optional[set]:
    if not left_column or not right_column:
        return None    
    return right_column.difference(left_column)

def number_of_products(column: Optional[set]) -> int:
    if not column:
        return 0
    return len(column)

grouped_texts_eans_per_month["receipt_text_intersection"] = grouped_texts_eans_per_month.apply(lambda row: intersection(row["receipt_text"], row["receipt_text_lagged"]), axis=1)
grouped_texts_eans_per_month["receipt_text_introduced"] = grouped_texts_eans_per_month.apply(lambda row: introduced_products(row["receipt_text"], row["receipt_text_lagged"]), axis=1)
grouped_texts_eans_per_month["number_receipt_text_introduced"] = grouped_texts_eans_per_month["receipt_text_introduced"].apply(number_of_products)
grouped_texts_eans_per_month["receipt_text_removed"] = grouped_texts_eans_per_month.apply(lambda row: removed_products(row["receipt_text"], row["receipt_text_lagged"]), axis=1)
grouped_texts_eans_per_month["number_receipt_text_removed"] = grouped_texts_eans_per_month["receipt_text_removed"].apply(number_of_products)

grouped_texts_eans_per_month["ean_intersection"] = grouped_texts_eans_per_month.apply(lambda row: intersection(row["ean_number"], row["ean_number_lagged"]), axis=1)
grouped_texts_eans_per_month["ean_introduced"] = grouped_texts_eans_per_month.apply(lambda row: introduced_products(row["ean_number"], row["ean_number_lagged"]), axis=1)
grouped_texts_eans_per_month["number_ean_introduced"] = grouped_texts_eans_per_month["ean_introduced"].apply(number_of_products)
grouped_texts_eans_per_month["ean_removed"] = grouped_texts_eans_per_month.apply(lambda row: removed_products(row["ean_number"], row["ean_number_lagged"]), axis=1)
grouped_texts_eans_per_month["number_ean_removed"] = grouped_texts_eans_per_month["ean_removed"].apply(number_of_products)

grouped_texts_eans_per_month

In [ ]:
lidl_2022_02_03 = grouped_texts_eans_per_month[(grouped_texts_eans_per_month.index  == "202202") | (grouped_texts_eans_per_month.index  == "202203")]
lidl_2022_02_03

In [ ]:
len(lidl_2022_02_03.iloc[0].receipt_text_introduced), len(lidl_2022_02_03.iloc[1].receipt_text_removed)

In [ ]:
receipt_text_different_02 = lidl_2022_02_03.iloc[0].receipt_text_introduced - lidl_2022_02_03.iloc[1].receipt_text_removed
receipt_text_different_02

In [ ]:
lidl_2022_02_03.iloc[0].receipt_text.intersection(receipt_text_different_02)

In [ ]:
eans_different_02 = set(lidl_df[lidl_df.receipt_text.isin(receipt_text_different_02)].ean_number)
eans_different_02 = set([int(float(ean)) for ean in eans_different_02])
eans_different_02

In [ ]:
len(lidl_2022_02_03.iloc[0].receipt_text_introduced - lidl_2022_02_03.iloc[1].receipt_text_removed)

In [ ]:
receipt_text_different_03 = lidl_2022_02_03.iloc[1].receipt_text_removed - lidl_2022_02_03.iloc[0].receipt_text_introduced
receipt_text_different_03

In [ ]:
lidl_2022_02_03.iloc[1].receipt_text.intersection(receipt_text_different_03)

In [ ]:
eans_different_03 = set(lidl_df[lidl_df.receipt_text.isin(receipt_text_different_03)].ean_number)
eans_different_03 = set([int(float(ean)) for ean in eans_different_03])
eans_different_03

In [ ]:
len(eans_different_03 - eans_different_02)

In [ ]:
len(lidl_2022_02_03.iloc[1].receipt_text_removed - lidl_2022_02_03.iloc[0].receipt_text_introduced)

In [ ]:
len(lidl_2022_02_03.iloc[0].receipt_text_introduced.intersection(lidl_2022_02_03.iloc[1].receipt_text_removed))

In [ ]:
same_texts_03 = lidl_2022_02_03.iloc[0].receipt_text_introduced.intersection(lidl_2022_02_03.iloc[1].receipt_text_removed)
sorted(same_texts)[2500:2550]

In [ ]:
same_texts_02 = lidl_2022_02_03.iloc[1].receipt_text_removed.intersection(lidl_2022_02_03.iloc[0].receipt_text_introduced)
same_texts_02

In [ ]:
os.getcwd()

In [ ]:
grouped_texts_eans_per_month[["number_receipt_text_introduced", "number_ean_introduced"]].to_latex("../Notebooks/tables/lidl_unique_receipt_texts_eans_introducted_per_month.tex")
grouped_texts_eans_per_month[["number_receipt_text_removed", "number_ean_removed"]].to_latex("../Notebooks/tables/lidl_unique_receipt_texts_eans_removed_per_month.tex")

In [ ]:
grouped_texts_eans_per_month[["number_receipt_text_introduced", "number_ean_introduced"]].plot(title="Number of unique texts/ean introduced per month").figure.savefig(os.path.join("../Notebooks/plots", "lidl_unique_receipt_texts_eans_introducted_per_month.png"))

In [ ]:
grouped_texts_eans_per_month[["number_receipt_text_removed", "number_ean_removed"]].plot(title="Number of unique texts/ean removed per month").figure.savefig(os.path.join("../Notebooks/plots", "lidl_unique_receipt_texts_eans_removed_per_month.png"))

In [ ]:

grouped_texts_per_month_coicop = lidl_new_products_df.groupby(by=["year_month", "coicop_level_1"])["receipt_text"].apply(series_to_set)
grouped_texts_per_month_coicop = grouped_texts_per_month_coicop.reset_index()
grouped_eans_per_month_coicop = lidl_new_products_df.groupby(by=["year_month", "coicop_level_1"])["ean_number"].apply(series_to_set)
grouped_eans_per_month_coicop = grouped_eans_per_month_coicop.reset_index()

grouped_texts_eans_per_month_coicop = grouped_texts_per_month_coicop.merge(grouped_eans_per_month_coicop, on=["year_month", "coicop_level_1"])
grouped_texts_eans_per_month_coicop["receipt_text_lagged"] = grouped_texts_eans_per_month_coicop["receipt_text"].shift(1)
grouped_texts_eans_per_month_coicop["ean_number_lagged"] = grouped_texts_eans_per_month_coicop["ean_number"].shift(1)

grouped_texts_eans_per_month_coicop["receipt_text_intersection"] = grouped_texts_eans_per_month_coicop.apply(lambda row: intersection(row["receipt_text"], row["receipt_text_lagged"]), axis=1)
grouped_texts_eans_per_month_coicop["receipt_text_introduced"] = grouped_texts_eans_per_month_coicop.apply(lambda row: introduced_products(row["receipt_text"], row["receipt_text_lagged"]), axis=1)
grouped_texts_eans_per_month_coicop["number_receipt_text_introduced"] = grouped_texts_eans_per_month_coicop["receipt_text_introduced"].apply(number_of_products)
grouped_texts_eans_per_month_coicop["receipt_text_removed"] = grouped_texts_eans_per_month_coicop.apply(lambda row: removed_products(row["receipt_text"], row["receipt_text_lagged"]), axis=1)
grouped_texts_eans_per_month_coicop["number_receipt_text_removed"] = grouped_texts_eans_per_month_coicop["receipt_text_removed"].apply(number_of_products)

grouped_texts_eans_per_month_coicop["ean_intersection"] = grouped_texts_eans_per_month_coicop.apply(lambda row: intersection(row["ean_number"], row["ean_number_lagged"]), axis=1)
grouped_texts_eans_per_month_coicop["ean_introduced"] = grouped_texts_eans_per_month_coicop.apply(lambda row: introduced_products(row["ean_number"], row["ean_number_lagged"]), axis=1)
grouped_texts_eans_per_month_coicop["number_ean_introduced"] = grouped_texts_eans_per_month_coicop["ean_introduced"].apply(number_of_products)
grouped_texts_eans_per_month_coicop["ean_removed"] = grouped_texts_eans_per_month_coicop.apply(lambda row: removed_products(row["ean_number"], row["ean_number_lagged"]), axis=1)
grouped_texts_eans_per_month_coicop["number_ean_removed"] = grouped_texts_eans_per_month_coicop["ean_removed"].apply(number_of_products)


grouped_texts_eans_per_month_coicop 

In [ ]:
for coicop_level in grouped_texts_eans_per_month_coicop.coicop_level_1.unique():
    grouped_texts_eans_per_month_coicop_level = grouped_texts_eans_per_month_coicop[grouped_texts_eans_per_month_coicop.coicop_level_1 == coicop_level]
    grouped_texts_eans_per_month_coicop_level[["year_month","number_receipt_text_introduced", "number_ean_introduced"]].plot(x="year_month", title=f"Number of introduced texts/eans per month for coicop {coicop_level}").figure.savefig(os.path.join("../Notebooks/plots", f"lidl_introduced_receipt_texts_eans_per_month_coicop_{coicop_level}.png"))

In [ ]:
for coicop_level in grouped_texts_eans_per_month_coicop.coicop_level_1.unique():
    grouped_texts_eans_per_month_coicop_level = grouped_texts_eans_per_month_coicop[grouped_texts_eans_per_month_coicop.coicop_level_1 == coicop_level]
    grouped_texts_eans_per_month_coicop_level[["year_month","number_receipt_text_removed", "number_ean_removed"]].plot(x="year_month", title=f"Number of removed texts/eans per month for coicop {coicop_level}").figure.savefig(os.path.join("../Notebooks/plots", f"lidl_removed_receipt_texts_eans_per_month_coicop_{coicop_level}.png"))

# Product Availability

In [ ]:
unique_receipt_texts = lidl_df.receipt_text.unique()
unique_receipt_texts

In [ ]:
len(unique_receipt_texts)

In [ ]:
unique_periods = lidl_df["year_month"].unique()
unique_periods

In [ ]:
from typing import List

def availability_in_period(dataframe: pd.DataFrame, unique_receipt_texts: List[str], period: str) -> List[bool]:
    period_texts = dataframe.loc[period]["receipt_text"]
    return [receipt_text in period_texts for receipt_text in unique_receipt_texts]

    
unique_receipt_texts = lidl_df.receipt_text.unique()
unique_periods = lidl_df["year_month"].unique()    

availability_per_period = {year_month: availability_in_period(grouped_texts_eans_per_month, unique_receipt_texts, year_month) for year_month in unique_periods}
availability_per_period["receipt_texts"] = unique_receipt_texts

product_availability_df = pd.DataFrame(availability_per_period)
product_availability_df = product_availability_df[["receipt_texts"] + unique_periods.tolist()]
product_availability_df.head()

In [ ]:
product_availability_sorted_df = product_availability_df.sort_values(by=product_availability_df.columns[1:].tolist())
product_availability_sorted_df

In [ ]:
month_columns = [column for column in product_availability_sorted_df.columns if column not in ["receipt_texts", "number_of_months_available"]]
month_columns

In [ ]:
total_number_of_months = len(month_columns)
total_number_of_months

In [ ]:
product_availability_sorted_df["number_of_months_available"] = product_availability_sorted_df[month_columns].sum(axis=1)
product_availability_sorted_df

In [ ]:
product_availability_sorted_df.plot.hist(column="number_of_months_available", bins=total_number_of_months).figure.savefig("../Notebooks/plots/lidl_product_lifetime_histogram.png")

In [ ]:
same_period_as_plus = [column for column in product_availability_sorted_df.columns if "20" in column and column not in [f"20210{month}" for month in range(7)] and int(column[:4]) >= 2021 ] + ["number_of_months_available"]
same_period_as_plus

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10,10))

lidl_product_available_plus_period = product_availability_sorted_df[same_period_as_plus].copy()
lidl_product_available_plus_period["number_of_months_available_plus"] = lidl_product_available_plus_period[same_period_as_plus[:-1]].sum(axis=1)
lidl_product_available_plus_period = lidl_product_available_plus_period[lidl_product_available_plus_period["number_of_months_available_plus"] > 0]

min_month, max_month = lidl_product_available_plus_period["number_of_months_available_plus"].min(), lidl_product_available_plus_period["number_of_months_available_plus"].max()
lidl_product_available_plus_period.plot.hist(column="number_of_months_available_plus", bins=len(same_period_as_plus[:-1]), ax=ax, range=(min_month,max_month)).figure.savefig("../Notebooks/plots/lidl_product_lifetime_histogram_plus_period.png")

In [ ]:
lidl_product_available_plus_period[lidl_product_available_plus_period["number_of_months_available_plus"] > 0]["number_of_months_available_plus"] .min()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

fig, ax = plt.subplots(figsize=(20,20))
sns.heatmap(product_availability_sorted_df[product_availability_sorted_df.columns[:-1]].set_index("receipt_texts"), cmap="tab20", ax=ax)
plt.savefig("../Notebooks/plots/lidl_products_over_time.png")

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

years = [year_month_columns[:4] for year_month_columns in product_availability_sorted_df.columns[1:]]

for year in years:
    fig, ax = plt.subplots(figsize=(20,20))
    
    year_columns = ["receipt_texts"] + [year_column for year_column in  product_availability_sorted_df.columns[1:] if year_column.startswith(year)]
    sns.heatmap(product_availability_sorted_df[year_columns].set_index("receipt_texts"), cmap="tab20", ax=ax)
    plt.savefig(f"../Notebooks/plots/lidl_products_over_time_{year}.png")

In [ ]:
new_products_202202 = product_availability_sorted_df[["receipt_texts", "202202"]][product_availability_sorted_df["202202"] == True]
new_products_202202

In [ ]:
from wordcloud import WordCloud

w = WordCloud().generate(" ".join([word for word in new_products_202202.receipt_texts]))
plt.imshow(w, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
import spacy

nlp = spacy.load("nl_core_news_md")

In [ ]:
lidl_new_products_df["features"] = [doc.vector for doc in nlp.pipe(lidl_new_products_df["receipt_text"], disable=["tagger", "parser", "ner"])]

In [ ]:
month = "201801"
period_df = lidl_new_products_df[lidl_new_products_df.month == month]
period_df.head()

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(period_df.features.values.tolist(), period_df.coicop_division.values.tolist())

In [ ]:
lidl_new_products_df.rename(columns={"month": "year_month"}, inplace=True)
lidl_new_products_df["year"] = lidl_new_products_df.year_month.str[:4].astype(int)
lidl_new_products_df["month"] = lidl_new_products_df.year_month.str[4:].astype(int)
lidl_new_products_df.head()

In [ ]:
from sklearn.exceptions import ConvergenceWarning
import warnings
import os


os.makedirs("lr_eval", exist_ok=True)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)

    eval_dfs = []
    periods = lidl_new_products_df["year_month"].unique()
    for period in periods[:-1]:
        year, month = int(period[:4]), int(period[4:])

        train_period_df = lidl_new_products_df[(lidl_new_products_df.year == year) & (lidl_new_products_df.month == month)]
        train_period_df.to_csv(os.path.join("lr_eval", f"train_df_{period}.csv"), index=False, sep=";")
    
    
        lr = LogisticRegression()
        lr.fit(train_period_df.features.values.tolist(), train_period_df.coicop_division.values.tolist())

        eval_period_df = lidl_new_products_df[((lidl_new_products_df.year == year) & (lidl_new_products_df.month > month) | (lidl_new_products_df.year > year))].copy()
        eval_period_df["y_pred"] = lr.predict(eval_period_df.features.values.tolist())
        eval_period_df.to_csv(os.path.join("lr_eval", f"eval_df_{period}.csv"), index=False, sep=";")
        
        eval_dfs.append(eval_period_df)
        del eval_period_df
        

In [ ]:
for eval_df in evals_dfs:
    eval_df.head()

In [ ]:
lidl_new_products_df["y_pred"] = lr.predict(lidl_new_products_df.features.values.tolist())
lidl_new_products_df.head()

In [ ]:
from sklearn.metrics import f1_score

f1_score(lidl_new_products_df.coicop_division, lidl_new_products_df.y_pred, average="macro")

In [ ]:
products_201801_df =  lidl_new_products_df[lidl_new_products_df.month == "201801"]
f1_score(products_201801_df.coicop_division, products_201801_df.y_pred, average="macro")

In [ ]:
products_201802_df =  lidl_new_products_df[lidl_new_products_df.month == "201802"]
old_products = products_201802_df[products_201802_df.new_text == False]
new_products = products_201802_df[products_201802_df.new_text == True]

f1_score(old_products.coicop_division, old_products.y_pred, average="macro"), f1_score(new_products.coicop_division, new_products.y_pred, average="macro")

In [ ]:
products_201802_df[(products_201802_df.new_text == True) & (products_201802_df.coicop_division == products_201802_df.y_pred)]

In [ ]:
products_201802_df[(products_201802_df.new_text == True) & (products_201802_df.coicop_division != products_201802_df.y_pred)]

In [ ]:
len(products_201802_df[(products_201802_df.new_text == True) & (products_201802_df.coicop_division != products_201802_df.y_pred)])

In [ ]:
283 / (283+26)

In [ ]:
products_201802_df[(products_201802_df.new_text == False) & (products_201802_df.coicop_division == products_201802_df.y_pred)]

In [ ]:
products_201802_df[(products_201802_df.new_text == False) & (products_201802_df.coicop_division != products_201802_df.y_pred)]